In [ ]:
import os
import openai
import autogen
from dotenv import load_dotenv, find_dotenv

openai.api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
config_list = {
    'model': 'gpt-4o-mini'
}

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

In [ ]:
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.assistant_agent import AssistantAgent

plot_data_agent = MultimodalConversableAgent(
    name = "Plot_data_retrieving_agent",
    max_consecutive_auto_reply = 10, 
    system_message = '''
        You are an image and json data extraction agent. Your task is to extract all plot data from the images with the aid of it's json file provided. 
        This includes identifying the x-axis, y-axis labels, titles, data points, and any other relevant details that describe the plot. 
        Do not ask for other information, and ensure that you capture all plot data clearly and accurately. 
        Return 'TERMINATE' once all plot data has been successfully extracted.
        ''',
    llm_config = {
        "config_list" : config_list, 
        "temperature" : 0.5, 
        "max_tokens" : 300
    }
)

plot_json_data_agent = MultimodalConversableAgent(
    name = "plot_json_data_retrieving_agent",
    max_consecutive_auto_reply = 10, 
    system_message = '''
        You are a json data extraction agent. Your task is to extract all plot data from the images with the aid of it's json file provided. 
        This includes identifying the x-axis, y-axis labels, titles, data points, and any other relevant details that describe the plot. 
        Do not ask for other information, and ensure that you capture all plot data clearly and accurately. 
        Return 'TERMINATE' once all plot data has been successfully extracted.
        ''',
    llm_config = {
        "config_list" : config_list, 
        "temperature" : 0.5, 
        "max_tokens" : 300
    }
)

plot_analysis_agent = AssistantAgent(
    name = "Plot_data_retrieving_agent",
    max_consecutive_auto_reply = 10, 
    system_message = '''
        You are an expert in analyzing time series plots. Your task is to analyze the provided image and json file of a time series plot and extract the key insights. 
        This includes identifying the time intervals, trends, peaks, valleys, outliers, and any seasonal or cyclical patterns. 
        Also, extract information about the axes (x-axis and y-axis), labels, and any relevant titles or legends. 
        Do not request any other information and provide a clear analysis of the time series data present in the plot. 
        Return 'TERMINATE' once the analysis is complete and all key insights have been extracted.
        ''',
    llm_config = {
        "config_list" : config_list, 
        "temperature" : 0.5, 
        "max_tokens" : 300
    }
)



In [ ]:
news_data_agent = MultimodalConversableAgent(
    name = "Plot_data_retrieving_agent",
    max_consecutive_auto_reply = 10, 
    system_message = '''
        You are an expert in extracting text data from news articles. Your task is to extract all the text content from the given article. 
        This includes headings, subheadings, paragraphs, quotes, dates, locations and any other relevant textual information within the article. 
        Do not ask for any additional details or information. Ensure that the entire text content is retrieved accurately and clearly. 
        Return 'TERMINATE' once all text data has been successfully extracted from the news article.
        '''
        ,
    llm_config = {
        "config_list" : config_list, 
        "temperature" : 0.5, 
        "max_tokens" : 300
    }
)

news_analysis_agent = AssistantAgent(
    name = "Plot_data_retrieving_agent",
    max_consecutive_auto_reply = 10, 
    system_message = '''
        You are an expert in analyzing news articles. Your task is to thoroughly analyze the provided news article and extract key insights. 
        This includes identifying the main topic, key points, important facts, quotes, and any conclusions or opinions presented. 
        Additionally, analyze the tone, sentiment, and writing style, and identify any biases or key arguments. 
        Do not request any further details, and provide a detailed analysis based on the content of the article. 
        Return 'TERMINATE' once the analysis is complete and all key insights have been extracted.
        '''
        ,
    llm_config = {
        "config_list" : config_list, 
        "temperature" : 0.5, 
        "max_tokens" : 300
    }
)

In [ ]:
supervisor = autogen.UserProxyAgent(
    name = "user_proxy",
    system_message='''You are an intermediary agent. 
        Your role is to relay user inputs to the appropriate agents and return their responses to the user. 
        You do not perform any tasks yourself but ensure smooth communication between the user and the system.''',
    is_termination_msg = termination_msg,
    human_input_mode = "TERMINATE",
    max_consecutive_auto_reply=0,
    code_execution_config={"use_docker": False}
)

In [ ]:
MESSAGE =  "Explain the graph using the image <img images/plot3.png>. and json data"

In [ ]:
def reset_agents():
    supervisor.reset(),
    plot_data_agent.reset(),
    plot_analysis_agent.reset(),
    news_data_agent.reset(),
    news_analysis_agent.reset(),


def main_chat():
    reset_agents()
    groupchat = autogen.GroupChat(
        agents=[supervisor, plot_data_agent, plot_analysis_agent, news_data_agent, news_analysis_agent],
        messages=[],
        max_round=12,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list)

    # Start chatting with the boss as this is the user proxy agent.
    supervisor.initiate_chat(
        manager,
        message=MESSAGE
    )

    # # Start chatting with the boss as this is the user proxy agent.
    # def send_image_and_text_to_agent(image_path, json_data):
    #     # You can load the image and json_data here, or if these are passed from another process
    #     # Convert image to a format (path or base64) that the agent can process, and pass both the image and data
    #     image_input = open(image_path, "rb").read()  # Read image as binary data
    #     text_input = json_data  # assuming json_data is the relevant time series data

    #     # Pass both image and text to the agent for processing
    #     supervisor.initiate_chat(
    #         manager=None,  # Manager can be defined as needed
    #         message=f"Analyze this plot: {image_input} with this data: {text_input}",
    #     )

    # # Simulated example of sending the image and text
    # send_image_and_text_to_agent("images/plot3.png", "images/data.json")

main_chat()

user_proxy (to chat_manager):

Explain the graph using the image <img images/plot3.png>. and json data  

--------------------------------------------------------------------------------

Next speaker: Plot_data_retrieving_agent


>>>>>>>> USING AUTO REPLY...
Plot_data_retrieving_agent (to chat_manager):

I can help you extract plot data from the graph and the JSON file provided. Please share the JSON file so I can assist you further.

--------------------------------------------------------------------------------

Next speaker: Plot_data_retrieving_agent


>>>>>>>> USING AUTO REPLY...
Plot_data_retrieving_agent (to chat_manager):

I can't analyze the graph directly, but I can help you extract plot data if you provide the JSON file associated with it. Please share that file!

--------------------------------------------------------------------------------

Next speaker: user_proxy

user_proxy (to chat_manager):

{     "GDP Growth Rate":[        {           "actualData":[              

In [ ]:
# message = """
#     what is on the image?
#     <img images/plot3.png>.
# """
# user_proxy.initiate_chat(plot_agent, message=message)       